# Woltka functional profiling

Command, coords, and Uniref map taken from Austins HNRC analyses scripts

## Re-run classification first
Old classification was done with previous version of WoL database that mapped to contigs. Need mapping to gOTU

In [11]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=48:00:00
#PBS -l nodes=1:ppn=24
#PBS -l mem=200gb
#PBS -N shogun_T1d

source ~/.bash_profile
conda activate shogun

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT

cd $tmp

out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/tax_profiles
log_file=$out_dir/logs/shogun_log.txt
db=/projects/wol/20170307/release/databases/shogun
fastq_file_list=/home/swandro/T1D/shogun/shotgun_seq_locations.tsv

while read id r1 r2
    do
        no_root=${id##*/}
        base_name=${no_root%.bowtie2.sam}
        if [ ! -f $out_dir/"$base_name"_bt2_wol_alignment.sam ]
            then
                start=`date +%s`
                cat $r1 $r2 | seqtk seq -A > $id.fasta
                shogun align \
                    -a bowtie2 \
                    -i $id.fasta \
                    -d $db \
                    -t 24 \
                    -o $tmp/
                end=`date +%s`
                runtime=$((end-start))
                mv alignment.bowtie2.sam $out_dir/"$base_name"_bt2_wol_alignment.sam
                echo Finished $id in $runtime seconds >> $log_file
            fi
    done < $fastq_file_list
' | qsub


1316273.barnacle.ucsd.edu


In [ ]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=48:00:00
#PBS -l nodes=1:ppn=24
#PBS -l mem=200gb
#PBS -N woltka_T1d

source ~/.bash_profile
conda activate woltka

coord_file=/projects/wol/20170307/prodigal/coords.txt.xz
uniref_map=/projects/wol/20170307/uniref/neo/uniref.map.xz

out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/profiles

cd /projects/cmi_proj/seed_grants/T1D_JaneKim/

for file in /home/swandro/T1D/shogun/*.sam
    do
        no_root=${file##*/}
        base_name=${no_root%.bowtie2.sam}
        if [ ! -f $out_dir/${base_name}_fprofile.txt ]
            then
                woltka classify -i $file \
                          --coords $coord_file \
                          --map $uniref_map \
                          --map-as-rank \
                          --rank uniref \
                          -o $out_dir/${base_name}_fprofile.txt
                echo Finished sample $base_name
            fi
    done
' | qsub

# Additional funcitonal profiling
used 145Gb  
UniRef, GO, KEGG, OrthoDB, PDB, RefSeq and MetaCyc

### Serial job

In [3]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=12:00:00
#PBS -l nodes=1:ppn=2
#PBS -l mem=200gb
#PBS -N test_woltka_T1d

source ~/.bash_profile
conda activate woltka

#Databases
db_root=/projects/wol/20170307/release/annotation
coord_file=$db_root/coords.txt.xz
uniref_map=$db_root/uniref.map.xz


out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/prof_testM
mkdir $out_dir
log_file=$out_dir/runtime_log.txt
cd /projects/cmi_proj/seed_grants/T1D_JaneKim/

file=/home/swandro/T1D/shogun/11129.NT1D019.JJK.St.bowtie2.sam 
    
no_root=${file##*/}
base_name=${no_root%.bowtie2.sam}

declare -A DBS=(
[process]=$db_root/go/process.tsv.xz
[ko]=$db_root/kegg/ko.map.xz
[orthodb]=$db_root/orthodb/orthodb.map.xz
)

for key in ${!DBS[@]}
    do
    echo Starting $key >> $log_file
    start=`date +%s`
    woltka classify -i $file \
              --coords $coord_file \
              --map $uniref_map \
              --map ${DBS[$key]} \
              --map-as-rank \
              --rank $key \
              -o $out_dir/NT1D019_profile_${key}
    end=`date +%s`
    runtime=$((end-start))
    echo Finished $key in $runtime seconds >> $log_file
    done


' | qsub

1320000.barnacle.ucsd.edu


In [1]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=12:00:00
#PBS -l nodes=1:ppn=2
#PBS -l mem=200gb
#PBS -N test_woltka_T1d

source ~/.bash_profile
conda activate woltka

#Databases
db_root=/projects/wol/20170307/release/annotation
coord_file=$db_root/coords.txt.xz
uniref_map=$db_root/uniref.map.xz


out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/prof_testU
mkdir $out_dir
log_file=$out_dir/runtime_log.txt
cd /projects/cmi_proj/seed_grants/T1D_JaneKim/

file=/home/swandro/T1D/shogun/11129.NT1D019.JJK.St.bowtie2.sam 
    
no_root=${file##*/}
base_name=${no_root%.bowtie2.sam}

declare -A DBS=(
[process]=$db_root/go/process.tsv.xz
[ko]=$db_root/kegg/ko.map.xz
[orthodb]=$db_root/orthodb/orthodb.map.xz
)

for key in ${!DBS[@]}
    do
    echo Starting $key >> $log_file
    start=`date +%s`
    woltka classify -i $file \
              --coords $coord_file \
              --map $uniref_map \
              --map ${DBS[$key]} \
              --map-as-rank \
              --rank uniref,$key \
              -o $out_dir/NT1D019_profile_${key}
    end=`date +%s`
    runtime=$((end-start))
    echo Finished $key in $runtime seconds >> $log_file
    done


' | qsub

1320645.barnacle.ucsd.edu


## Array Job

In [1]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=12:00:00
#PBS -l nodes=1:ppn=2
#PBS -l mem=200gb
#PBS -N test_woltka_T1d

source ~/.bash_profile
conda activate woltka

#Databases
db_root=/projects/wol/20170307/release/annotation
coord_file=$db_root/coords.txt.xz
uniref_map=$db_root/uniref.map.xz


out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/prof_test
log_file=$out_dir/array_runtime_log.txt
cd /projects/cmi_proj/seed_grants/T1D_JaneKim/

file=/home/swandro/T1D/shogun/11129.NT1D019.JJK.St.bowtie2.sam 
    
no_root=${file##*/}
base_name=${no_root%.bowtie2.sam}


keys=(go_map kegg_map ortho_map)
key=${keys[$PBS_ARRAYID]}
          
declare -A DBS=(
[go_map]=$db_root/go/process.tsv.xz
[kegg_map]=$db_root/kegg/ko.map.xz
[ortho_map]=$db_root/orthodb/orthodb.map.xz
)

start=`date +%s`
woltka classify -i $file \
          --coords $coord_file \
          --map $uniref_map \
          --map ${DBS[$key]} \
          --map-as-rank \
          --rank uniref_${key},$key \
          -o $out_dir/NT1D019_profile_${key}
end=`date +%s`
runtime=$((end-start))
echo Finished $key in $runtime seconds >> $log_file



' | qsub -t 0-2

1316222[].barnacle.ucsd.edu


In [15]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=72:00:00
#PBS -l nodes=1:ppn=2
#PBS -l mem=200gb
#PBS -N woltka_T1d

source ~/.bash_profile
conda activate woltka

#Databases
db_root=/projects/wol/20170307/release/annotation
coord_file=$db_root/coords.txt.xz
uniref_map=$db_root/uniref.map.xz


out_root=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/profiles2
log_file=$out_root/logs/woltka_functional_profiling_log.txt
cd $out_root

declare -A DBS=(
[go_map]=$db_root/go/process.tsv.xz
[kegg_map]=$db_root/kegg/ko.map.xz
[ortho_map]=$db_root/orthodb/orthodb.map.xz
)


for file in /projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/tax_profiles/*.sam
    do
    no_root=${file##*/}
    base_name=${no_root%.bowtie2.sam}
    out_dir=${out_root}/${base_name}
    if [ ! -d $out_dir ]
        then
        mkdir $out_dir
        for key in ${!DBS[@]}
            do   
                start=`date +%s`
                woltka classify -i $file \
                      --coords $coord_file \
                      --map $uniref_map \
                      --map ${DBS[$key]} \
                      --map-as-rank \
                      --rank uniref_${key},$key \
                      -o $out_dir/$key
                end=`date +%s`
                runtime=$((end-start))
                echo Finished $key in $runtime seconds >> $log_file
            done
        fi
    done
    
' | qsub

1316932.barnacle.ucsd.edu


## Test if Uniref on original sams works

In [1]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=12:00:00
#PBS -l nodes=1:ppn=24
#PBS -l mem=200gb
#PBS -N woltka_T1d_uniref

source ~/.bash_profile
conda activate woltka

db_root=/projects/wol/20170307/release/annotation
coord_file=$db_root/coords.txt.xz
uniref_map=$db_root/uniref.map.xz

out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/profiles_uniref_og
mkdir -p $out_dir
log_dir=$out_dir/logs
mkdir -p $out_dir/logs

cd /projects/cmi_proj/seed_grants/T1D_JaneKim/

for file in /home/swandro/T1D/shogun/*.sam
    do
        no_root=${file##*/}
        base_name=${no_root%.bowtie2.sam}
        if [ ! -f $out_dir/${base_name}_fprofile.txt ]
            then
                woltka classify -i $file \
                          --coords $coord_file \
                          --map $uniref_map \
                          --map-as-rank \
                          --rank uniref \
                          -o $out_dir/${base_name}_fprofile.txt
                echo Finished sample $base_name >> $log_dir/log.txt
            fi
    done
' | qsub

1318532.barnacle.ucsd.edu


## Test if Uniref on new sams works

In [2]:
%%script bash

echo '

#!/bin/bash -l
#PBS -m ae
#PBS -M swandro@ucsd.edu
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=12:00:00
#PBS -l nodes=1:ppn=24
#PBS -l mem=200gb
#PBS -N woltka_T1d_uniref_new

source ~/.bash_profile
conda activate woltka

db_root=/projects/wol/20170307/release/annotation
coord_file=$db_root/coords.txt.xz
uniref_map=$db_root/uniref.map.xz

out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/profiles_uniref_new2
mkdir -p $out_dir
log_dir=$out_dir/logs
mkdir -p $out_dir/logs

cd /projects/cmi_proj/seed_grants/T1D_JaneKim/

for file in /projects/cmi_proj/seed_grants/T1D_JaneKim/shogun_function/tax_profiles/*.sam
    do
        no_root=${file##*/}
        base_name=${no_root%.bowtie2.sam}
        if [ ! -f $out_dir/${base_name}_fprofile.txt ]
            then
                woltka classify -i $file \
                          --coords $coord_file \
                          --map $uniref_map \
                          --map-as-rank \
                          --rank uniref \
                          -o $out_dir/${base_name}_fprofile.txt
                echo Finished sample $base_name >> $log_dir/log.txt
            fi
    done
' | qsub

1319990.barnacle.ucsd.edu
